In [1]:
from sklearn import datasets
from time import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Importing our own
from models.trainer import TrainModelWrapper
from models.classification import Classification, Classification_Dropout
from models.regression import Regression, Regression_Dropout
import utils.datasets as DB

## Classification

In [2]:
datasets = DB.Datasets()
train_mnist, test_mnist = datasets.get_MNIST()
train_mnist, val_mnist = torch.utils.data.random_split(train_mnist, [50000, 10000])

In [3]:
# Multi Class Problem
class NeuralNetwork(nn.Module):
    
    def __init__(self,input_dim,hidden_nodes, num_classes, dropout=False):
        super(NeuralNetwork,self).__init__()
        
        self.input_dim = input_dim
        self.dropout = dropout
        self.linear1 = nn.Linear(input_dim,hidden_nodes)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_nodes,hidden_nodes)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_nodes,num_classes)
        if dropout:
            self.drop1 = nn.Dropout(0.2)
            self.drop2 = nn.Dropout(0.5)
            self.drop3 = nn.Dropout(0.5)
    
    def forward(self,x):
        x = x.view(-1, self.input_dim)
        if self.dropout:
            x = self.drop1(x)
        out = self.linear1(x)
        
        if self.dropout:
            out = self.drop2(out)
        out = self.relu1(out)
        out = self.linear2(out)
        
        if self.dropout:
            out = self.drop3(out)
        out = self.relu2(out)
        out = self.linear3(out)
        # NO SOFTMAX since done by crossentropyloss
        return out

In [4]:
def train_save_classification(nodes_hidden, learning_rate, name, dropout=False):
    num_epochs = 600
    batch_size = 128
    
    if not dropout:
        model = Classification(input_dim=784, output_dim=10, hl_units=nodes_hidden, hl_type=nn.Linear)
    else:
        model = Classification_Dropout(input_dim=784, output_dim=10, hl_units=nodes_hidden, hl_type=nn.Linear)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    
    args = {
            'model': model,
            'model_name':name,
            'train_dataset': train_mnist,
            'criterion': criterion,
            'batch_size': batch_size,
            'optimizer': optimizer,
            'es_flag': False,
            'num_epochs': num_epochs,
            'val_dataset': val_mnist,
            'test_dataset': test_mnist,
            'mode': 2
            }
    trainer = TrainModelWrapper(**args)
    model, history = trainer.train()
    
    torch.save(model.state_dict(), 'weights/'+name+'.pth')
    return model, history, trainer
    

In [ ]:
lrs = [1e-3, 1e-4, 1e-5]
layers = [400, 800, 1200]

for lr in lrs:
    for hidden_size in layers:
        _name = f"{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_classification(nodes_hidden=hidden_size, learning_rate=lr, name=_name)
        
for lr in lrs:
    for hidden_size in layers:
        _name = f"drop_{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_classification(nodes_hidden=hidden_size, learning_rate=lr, name=_name, dropout=True)


2023-02-24 20:40:42,639 [WARNING] Scheduler Object not found amongst the Arguments. Ignore warning if scheduler wasn't meant to be in the loop


400_1e-03


2023-02-24 20:40:44,464 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-02-24 20:40:44,465 [INFO] NumExpr defaulting to 8 threads.


The tensorboard model name corresponding to this model is 400_1e-03_20230224204042639093
Epoch 1/600
----------


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:06<00:00, 58.44it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:05<00:00, 68.58it/s]


train Loss: 0.5755
train Acc: 0.0857


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 73.90it/s]


val Loss: 0.5755
val Acc: 0.0890


  0%|                                                                                           | 0/79 [00:00<?, ?it/s]

In [ ]:
# Loading the model again and continuing training is possible
"""
new_model = MLPModel(input_dim=784, output_dim=10, hidden_layer=400)
new_model.load_state_dict(torch.load('weights/400_1e-03.pth'))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(new_model.parameters(),lr=1e-3)

args = {
        'model': new_model,
        'train_dataset': train_mnist,
        'criterion': criterion,
        'batch_size': 128,
        'optimizer': optimizer,
        'es_flag': False,
        'num_epochs': 10,
        'val_dataset': val_mnist,
        'mode': 2
        }
trainer = TrainModelWrapper(**args)
model, history = trainer.train()
"""

## Regression

In [ ]:
datasets = DB.Datasets()
train_regr, test_regr = datasets.get_regression()

In [ ]:
#plt.scatter(x=np.array(train_regr)[:,0], y=np.array(train_regr)[:,1], color='r')
#plt.scatter(x=np.array(test_regr)[:,0], y=np.array(test_regr)[:,1], color='b')

In [ ]:
def train_save_regression(nodes_hidden, learning_rate, name, dropout=False):
    num_epochs = 600
    batch_size = 128
    
    if not dropout:
        model = Regression(input_dim=1, output_dim=1, hl_units=nodes_hidden, hl_type=nn.Linear)
    else:
        model = Regression_Dropout(input_dim=1, output_dim=1, hl_units=nodes_hidden, hl_type=nn.Linear)
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    
    args = {
            'model': model,
            'model_name': name,
            'train_dataset': train_regr,
            'criterion': criterion,
            'batch_size': batch_size,
            'optimizer': optimizer,
            'es_flag': False,
            'num_epochs': num_epochs,
            'val_dataset': test_regr,
            'test_dataset': test_regr,
            'mode': 0
            }
    trainer = TrainModelWrapper(**args)
    model, history = trainer.train()
    
    torch.save(model.state_dict(), 'weights/'+name+'.pth')
    return model, history, trainer

In [ ]:
lrs = [1e-3, 1e-4, 1e-5]
layers = [400, 800, 1200]

for lr in lrs:
    for hidden_size in layers:
        _name = f"regr_{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_regression(nodes_hidden=hidden_size, learning_rate=lr, name=_name)
        
for lr in lrs:
    for hidden_size in layers:
        _name = f"regr_drop_{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_regression(nodes_hidden=hidden_size, learning_rate=lr, name=_name, dropout=True)

In [ ]:
"""
m, h, t = train_save_classification(
    nodes_hidden=800, learning_rate=1e-3, name="test_class", dropout=True
)
predc = t.predict(test_mnist, out_dim=10)


m, h, t = train_save_regression(nodes_hidden=400, learning_rate=1e-3, name="test")
predr = t.predict(test_regr, out_dim=1)
"""